In [132]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from openai import OpenAI
import nest_asyncio
nest_asyncio.apply()
import asyncio  # Add this import
import os
from dotenv import load_dotenv
import json
from APIs.combinedapi import PubMedProcessor
%autoawait asyncio
import datetime



openai_client = OpenAI(api_key="OPENAI_API_KEY")


load_dotenv()

llm = ChatOpenAI()

llm = ChatOpenAI(api_key="OPENAI_API_KEY" )



In [118]:
chatgpt_prompt = """# IMPORTANT:
Try to answer this question/instruction with step-by-step thoughts and make the answer more structural.
Use `\n\n` to split the answer into several paragraphs.
Just respond to the instruction directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.
'''Develop a scientific rationale for the following:
                             
                             You are a AI assistant with a background in drug discovery.

    Given target: Gamma secretase
    Given disease: Alzheimer's disease
    Given mode of action: Gamma secretase is a multi-subunit protease complex that cleaves type I transmembrane proteins, including the amyloid precursor protein (APP) leading to the generation of amyloid-beta (Aβ) peptides.

    Context:
     Aβ is a family of secreted peptides generated from the sequential cleavages of the type 1 membrane protein APP by beta-secretase (BACE) and gamma-secretase (GSEC), respectively. 
      BACE cleaves APP in the luminal domain, releasing the N-terminal soluble APPβ domain and leaving the C-terminal fragment, APP-CTF, which remains in the membrane. 
      Subsequently, the APP-CTF is recruited to GSEC, a complex comprising four subunits, including PS, which harbors the active site. GSEC first cuts APP-CTF at the epsilon-cleavage site located close to the inner leaflet of the membrane. 
      This cleavage event produces either Aβ48 or Aβ49 and the APP intracellular domain (AICD). The membrane-retained Aβ48 or Aβ49 is then further processed by GSEC in a continuous cascade of proteolytical events at every third of fourth amino acid, where the N-terminal product of each reaction becomes the substrate for the next GSEC cleavage event.
       Accordingly, GSEC processes APP-CTF along two main product lines, Aβ49 → 46 → 43 → 40 → 37… and Aβ48 → 45 → 42 → 38…, respectively (Takami et al., 2009; Matsumura et al., 2014; Olsson et al., 2014). During this processing cascade, Aβ43 and shorter Aβ peptides stochastically escape further processing by GSEC and are released into the extracellular space. 
      As a result, Aβ peptides varying from 30 to 43 amino acids in length are secreted into the extracellular space. Among all secreted Aβ, Aβ40 is the most abundant in human CSF, followed by Aβ38, Aβ42, and Aβ37 (Liu et al., 2022). In cognitively normal individuals, Aβ42 and Aβ43 represent a smaller portion of the total secreted Aβ (Liu et al., 2022).
         These longer forms of Aβ seed the formation of Aβ-amyloid aggregates, a key step in the formation of amyloid plaques (Veugelen et al., 2016), as illustrated in Figure 1. Aβ42, which is produced in higher amounts than Aβ43, is the most abundant Aβ in amyloid plaques (Welander et al., 2009).


    Task 1: Develop a scientific rationale for Gamma secretase in Alzheimer's disease.

    Highlight the working hypothesis for the clinical target rationale and human biology evidence by minimum 2000 words.

    Describe as much as possible the evidence in humans or in human tissue that link the target, target space or approach to the pathogenesis of interest.
    If known, also describe here the wanted mode of action with regards to desired clinical outcome.
    Please avoid including only pre-clinical data in this section.

    Use the following structure and provide a detailed description for each point:
    - Working hypothesis:
    - Create a detailed description of the following idea: Develop an inhibitor of the neurokinin 3 receptor (NK3R) GPCR for the treatment of vasomotor symptoms due to menopause.
    - Is there are significant unmet medical need?
    - Is it suitable for combination therapy?
    - Which predictive biomarkers exist for the target related to the disease?
        - Provide a detailed description of existing clinical relevant biomarkers.

    - Clinical target rationale:
    - How relevant is the target location to the disease biology?
    - How it the target expression altered in human disease?
    - How is the target involved in the physiological process relevant to the disease?
    - Which phenotypes and genotypes were identified for the target?
    - How is the genetic link between the target and the disease?
    - Describe the evidence provided in clinics or by tools acting on the pathway where the target is involved.
    - Which kind of target modulation is required to treat the disease?

    - Challenges for the drug discovery program related to the target.
    - Check the following idea for details on small molecule compounds: Develop an inhibitor of the neurokinin 3 receptor (NK3R) GPCR for the treatment of vasomotor symptoms due to menopause..
    - Is a 'information driven approach' (IDA) strategy based on available small molecules possible?
        - Which small molecular modulators of the target known?
        - Which inhibitors, antagonists, agonists, negative allosteric modulators (NAM), positive allosteric modulators (PAM) are required for target modulation in the given disease? 
    - Which patients would respond the therapy?
    - Is the proposed mode of action on the target desirable and commercially viable in a clinical setting?
    - What are advantages and disadvantages of different therapeutic modalities (antibodies, small molecules, antisense oligonucleotides, PROTACs, molecular glue, peptide macrocycles, and so on) for tackling the target?

    - Alternative indications:
    - Describe alternative indication for modulators of the target and explain why."""


scientific_rationale_system_prompt = """You are 'scientific rationale', an expert in target discovery and assessment at a pharmaceutical company, you get commission when you assess and discovery a new target. 
                        You provide scientific rationales to support the selection of drug targets for a specific disease.
                        You are meticulous in your analysis and provide evidence-based assessments to guide drug discovery efforts.
                        Your role is to:
                        1. Your primary role is to develop comprehensive scientific rationales for given targets in specified diseases using available literature. """


refine_prompt ='''
Referencing the answers provided by all agents, synthesize a more detailed and comprehensive response by integrating all relevant details from these answers. Ensure logical coherence and provide ONLY THE MERGED ANSWER AS THE OUTPUT, omitting any discussion of the comparison process or analytical thoughts.
'''

In [137]:
import json
from typing import Dict, Any
import openai
import asyncio
from typing import Dict, Any


async def pubmed_paperqa(query: str) -> Dict[str, Any]:
    """ Searches PubmedCentral for papers using a query
    and returns the most relevant chunks using paperQA"""
    
    max_results: int = 10
    pubmed_query = query
    doc_query = query
    email = "sanazkazemi@hotmail.com"
    print(f"pubmed_paperqa called with query: {query}, max_results: {max_results}")
    
    pubmed_instance = PubMedProcessor(email)
    results_dict = await pubmed_instance.full_process(pubmed_query, doc_query, max_results)
    
    return json.dumps(results_dict, indent=4)

# To run this in a Jupyter notebook cell:
# query = "Alzheimer's disease and gamma secretase"
# results = await pubmed_paperqa(query)
# print(results)





def pick_best_query(queries: list, question: str) -> str:
    """picks the best query from the list of queries"""

    gpt_prompt ="""
    for the given question, pick the best query 
    from the list of queries that you think is most relevant to the question.
    ## IMPORTANT:
    Just return the best query. Do not add any additional information.
    """

    best_query = openai.chat.completions.create(

     model = "gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "you are a scientific researcher, you are tasked with finding the best query to search for scientific papers on PubMed."            
        },
            {
                "role": "user",
                "content": f"##Instruction:{gpt_prompt}\n\n###Question: {question}\n\n##Queries: {[queries]}\n\n"
            }
        ],
        temperature=0.7 # here you can adjust the temperature to get more or less creative search terms
    ).choices[0].message.content

    return best_query







def get_query(question, answer, num_queries) -> str:

    """Generates queries to search for in Pubmed based on the question"""    
    query_prompt = """ You are a scientific researcher, 
                        you are tasked with finding the best query to search 
                        for scientific papers on PubMed.
                                                    
                            I want to verify the content correctness of the given question, especially the last sentences.
                            Please summarize the content with the corresponding question.
                            This summarization will be used as a query to search with Bing search engine.
                            The query should be short but need to be specific to promise Bing can find related knowledge or pages.
                            You can also use search syntax to make the query short and clear enough for the search engine to find relevant language data.
                            Try to make the query as relevant as possible to the last few sentences in the content.
                            **IMPORTANT**
                            Just output the query directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.

                        The following worked very well for me in the past in terms of generating the highest number of results use it as a guide:
                        ###Example:
                        "{Target}" AND "{Disease}" AND ("{relevant_keyword}" OR "{relevant_keyword_1} Or "{relevant_keyword_n}")" and so on.
                        ##IMPORTANT:
                        Just provide the query. Do not add any additional information.
                        DO NOT copy the given example"""
    

    queries = []

    for i in range(num_queries):
        try:
            query = openai.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "system",
                        "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."

                    },
                    {
                        "role": "user",
                        "content": f"##Question: {question}\n\n##Content: {answer}\n\n##Instruction: {query_prompt}"
                    }
                ],
                temperature=1
            ).choices[0].message.content
            print(f"query {i}: {query}")

        except Exception as e:
            print(f"error {e}")
        queries.append(query)

    best_query = pick_best_query(queries, question)
    print(f"best query: {best_query}")

    return best_query




async def main(question: str, answer, num_queries: int):
    """Main function to get the best query for the question"""
    
    
    search_query = get_query(question, answer, num_queries)
    
    # Remove only the outermost single quotes if they exist otherwise doesnt work - not elegant but works
    if search_query.startswith("'") and search_query.endswith("'"):
        cleaned_query = search_query[1:-1]
    else:
        cleaned_query = search_query
    
    # Replace escaped single quotes with regular single quotes
    cleaned_query = cleaned_query.replace("\\'", "'")
    
    print(f"Cleaned search query: {cleaned_query}")
    
    results = await pubmed_paperqa(cleaned_query)
    return results


In [162]:
answer = """ ### Working Hypothesis

The working hypothesis for targeting gamma secretase (GSEC) in Alzheimer's disease (AD) is based on the premise that the dysregulation of amyloid-beta (Aβ) peptide production, particularly the accumulation of neurotoxic forms such as Aβ42 and Aβ43, is a central pathogenic mechanism in AD. Inhibition of GSEC could reduce the production of these longer, aggregation-prone Aβ peptides, thereby potentially preventing or slowing the formation of amyloid plaques, which are hallmarks of AD pathology.

### Unmet Medical Need

AD represents a significant unmet medical need, with millions of individuals affected globally and no disease-modifying treatments available. Current therapies primarily address symptomatic relief but do not halt or reverse the underlying disease process. The need for effective interventions that target the amyloid cascade hypothesis, particularly through modulation of GSEC activity, is critical to altering the disease trajectory.

### Suitability for Combination Therapy

Targeting GSEC may be suitable for combination therapy with other approaches aimed at reducing Aβ aggregation or enhancing clearance mechanisms, such as monoclonal antibodies targeting Aβ or therapies enhancing microglial function. Combination strategies could provide a multifaceted approach to addressing the complex pathophysiology of AD.

### Predictive Biomarkers

Existing clinical biomarkers related to GSEC and Aβ metabolism include:

- **Amyloid PET Imaging**: This imaging technique allows for the visualization of amyloid plaques in the brain, providing evidence of the pathological burden of Aβ in living patients.
  
- **CSF Aβ Levels**: Measurement of Aβ42 and Aβ40 in cerebrospinal fluid (CSF) is clinically relevant, as a decrease in Aβ42 relative to Aβ40 is indicative of amyloid pathology.

- **Neurodegeneration Biomarkers**: Markers such as phosphorylated tau (p-tau) in CSF and neurofilament light chain (NfL) can provide insights into neurodegeneration processes linked to GSEC activity and Aβ accumulation.

### Clinical Target Rationale

#### Relevance of Target Location to Disease Biology

GSEC is located in the membrane of neurons and glial cells, where it plays a crucial role in the cleavage of APP. This positioning is highly relevant to AD biology, as Aβ peptides generated by GSEC are central to the formation of amyloid plaques, a key pathological feature of AD.

#### Altered Target Expression in Human Disease

Studies have shown that GSEC activity may be dysregulated in AD, leading to increased production of toxic Aβ species. Additionally, post-mortem analyses of AD brains have indicated altered expression levels of GSEC subunits, which may contribute to the pathological processing of APP.

#### Involvement in Physiological Processes

GSEC is involved in the physiological processing of APP and other type I transmembrane proteins. Its cleavage activity not only generates Aβ but also produces signaling molecules such as the APP intracellular domain (AICD), which may have neuroprotective roles. The balance of these processes is crucial for maintaining neuronal health.

#### Identified Phenotypes and Genotypes

Genetic studies have identified several mutations in the APP, PSEN1, and PSEN2 genes, which encode components of the GSEC complex. These mutations are associated with familial forms of AD, underscoring the genetic link between GSEC activity and AD pathogenesis.

#### Evidence from Clinical Studies

Clinical evidence supporting the role of GSEC in AD includes the efficacy of GSEC inhibitors in preclinical models and the observation that modulation of Aβ levels correlates with cognitive outcomes. Furthermore, ongoing clinical trials targeting GSEC provide additional insights into the potential therapeutic benefit of this approach.

#### Required Target Modulation

To effectively treat AD, modulation of GSEC should ideally aim to reduce the production of neurotoxic Aβ peptides while preserving the cleavage of other substrates necessary for neuronal function. A fine balance must be struck to achieve therapeutic efficacy without compromising physiological processes.

### Challenges for Drug Discovery Program

Challenges in developing GSEC inhibitors include the complexity of the GSEC multi-subunit structure, potential off-target effects due to the involvement of GSEC in the processing of various substrates, and the need for specificity to avoid disrupting normal physiological functions.

### Information Driven Approach (IDA)

An information-driven approach for developing small molecule inhibitors of GSEC is feasible, leveraging existing knowledge of the GSEC complex and its role in Aβ production.

#### Known Small Molecular Modulators

Several small molecular modulators of GSEC have been identified, including:

- **GSEC Inhibitors**: Compounds that selectively inhibit GSEC activity have shown promise in preclinical studies, demonstrating reduced Aβ production.

- **Positive Allosteric Modulators (PAMs)**: These may enhance the cleavage of non-amyloidogenic substrates while inhibiting Aβ production.

#### Required Modulators for Target Modulation

Inhibitors and antagonists are necessary for reducing Aβ production, while PAMs may be explored to fine-tune GSEC activity. Negative allosteric modulators (NAMs) could also be beneficial in modulating GSEC activity in a more controlled manner.

### Patient Response to Therapy

Patients with AD who exhibit elevated levels of Aβ42 and Aβ43 or those with genetic predispositions linked to GSEC dysregulation may be more likely to respond to GSEC-targeted therapies.

### Desirability and Commercial Viability

The proposed mode of action targeting GSEC is desirable, as it addresses a core pathological mechanism in AD. Commercial viability is supported by the large patient population and the significant unmet need for disease-modifying therapies.

### Advantages and Disadvantages of Therapeutic Modalities

- **Antibodies**: High specificity and potential for targeting aggregated forms of Aβ, but may not penetrate the blood-brain barrier effectively.

- **Small Molecules**: Potential for oral bioavailability and better CNS penetration, but may face challenges with selectivity and off-target effects.

- **Antisense Oligonucleotides**: Target specific mRNA for Aβ production, but delivery to the CNS remains a challenge.

- **PROTACs**: Offer a novel mechanism for targeted protein degradation, but complexity in design and delivery may hinder development.

- **Molecular Glue and Peptide Macrocycles**: Emerging modalities with potential for selective targeting, though still in early stages of research.

### Alternative Indications

Potential alternative indications for GSEC modulators could include other neurodegenerative diseases characterized by protein aggregation, such as frontotemporal dementia or Parkinson's disease. The rationale for these indications lies in the shared pathological mechanisms involving protein misfolding and aggregation, suggesting that GSEC modulation may have broader therapeutic implications beyond AD.########################################
Processing draft..."""


results = await main(chatgpt_prompt, answer, num_queries=5)
print(results)


query 0: "Gamma secretase" AND "Alzheimer's disease" AND ("amyloid-beta production" OR "neurodegenerative diseases" OR "protein aggregation")
query 1: "Gamma secretase" AND "Alzheimer's disease" AND ("Aβ42" OR "Aβ43" OR "amyloid plaques" OR "GSEC inhibitors")
query 2: "Gamma secretase" AND "Alzheimer's disease" AND ("protein aggregation" OR "frontotemporal dementia" OR "Parkinson's disease")
query 3: "Gamma secretase" AND "Alzheimer's disease" AND ("predictive biomarkers" OR "clinical target rationale" OR "target modulation")
query 4: "Gamma secretase" AND "Alzheimer's disease" AND ("neurodegenerative diseases" OR "protein aggregation" OR "Parkinson's disease" OR "frontotemporal dementia")
best query: '"Gamma secretase" AND "Alzheimer\'s disease" AND ("Aβ42" OR "Aβ43" OR "amyloid plaques" OR "GSEC inhibitors")'
Cleaned search query: "Gamma secretase" AND "Alzheimer's disease" AND ("Aβ42" OR "Aβ43" OR "amyloid plaques" OR "GSEC inhibitors")
pubmed_paperqa called with query: "Gamma secre

In [148]:
num_agents = 3
num_steps = 3
final_output_mode = 'only_last_step'


In [149]:
import openai

def COT_agent(question):
    """Generates a chain of thought answer for comparison to RATT.
    question: str: the prompt to answer
    draft_prompt: str: the prompt to generate the draft
    system_prompt: str: the prompt to generate the system message"""

    draft_prompt = '''
IMPORTANT:
Try to answer this question/instruction with step-by-step thoughts and make the answer more structured.
Use `\n\n` to split the answer into several paragraphs.
Just respond to the instruction directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.
'''

    # Loop to generate different initial answers
    COT_draft = openai.chat.completions.create(
         model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": scientific_rationale_system_prompt # defined at the top
                },
                {
                    "role": "user",
                    "content": question + draft_prompt
                }
            ],
            temperature=0.5
        ).choices[0].message.content

    return COT_draft


In [179]:
from datetime import datetime


def split_draft(draft, split_char='\n\n'):
    # split_char: '\n\n'
    draft_paragraphs = draft.split(split_char)
    # print(f"The draft answer has {len(draft_paragraphs)}")
    return draft_paragraphs


def get_revise_answer(question, answer, retrieved_data):
    revise_prompt = '''
I want to revise the answer according to retrieved related text of the question in WIKI pages.
You need to check whether the answer is correct.
If you find some errors in the answer, revise the answer to make it better.
If you find some necessary details are ignored, add it to make the answer more plausible according to the related text.
If you find that a part of the answer is correct and does not require any additional details, maintain that part of the answer unchanged. Directly output the original content of that part without any modifications.
**IMPORTANT**
Try to keep the structure (multiple paragraphs with its subtitles) in the revised answer and make it more structual for understanding.
Split the paragraphs with `\n\n` characters.
Just output the revised answer directly. DO NOT add additional explanations or annoucement in the revised answer unless you are asked to.
'''
    revised_answer = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
            },
            {
                "role": "user",
                "content": f"##Pubmed central retrieved articles: {retrieved_data}\n\n##Question: {question}\n\n##previous Answer: {answer}\n\n##Instruction: {revise_prompt}"
            }
        ],
        temperature=0.5
    ).choices[0].message.content
    
    
    return revised_answer



async def RAG(question, draft_paragraphs):
    """ args:
    question: str: the prompt to answer
    draft_paragraphs: list: the list of paragraphs from the initial n drafts
    """
    answer = ""

    for i, paragraph in enumerate(draft_paragraphs):
        answer += '\n\n' + paragraph

        api_response = await main(question, answer, num_queries=2)  # Now using the entire answer instead of just the paragraph

        revised_answer = get_revise_answer(question, answer, api_response)  # Using the entire answer
        if revised_answer:
            answer = revised_answer  # Update the entire answer
        
        print(f"Completed iteration {i+1}/{len(draft_paragraphs)}")

        print('+'* 80 + '\n\n')
        print(f"RESULT OF PUBMED API:\n{answer}")
    
    return answer



def split_draft(draft, split_char='\n\n'):
    # split_char: '\n\n'
    draft_paragraphs = draft.split(split_char)
    # print(f"The draft answer has {len(draft_paragraphs)}")
    return draft_paragraphs



def get_draft_tot_initial(question: str, num_agents: int):
    """Generates initial answers from multiple agents for comparison"""
    draft_prompt = """
            IMPORTANT:
            Try to answer this question/instruction with step-by-step thoughts and make the answer more structured.
            Use `\n\n` to split the answer into several paragraphs.
            Just respond to the instruction directly. DO NOT add additional explanations or introducement in the answer unless you are asked to.
            """

    refine_prompt = """
            Referencing the answers provided by all agents, synthesize a more detailed and comprehensive response by integrating all relevant details from these answers. 
            Ensure logical coherence and provide ONLY THE MERGED ANSWER AS THE OUTPUT, omitting any discussion of the comparison process or analytical thoughts."""

    agent_drafts = []
    for i in range(num_agents):
        draft = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
                },
                {
                    "role": "user",
                    "content": question + draft_prompt
                }
            ],
            temperature=0.5
        ).choices[0].message.content
        print(f"####################draft {i}: {draft}########################################")

        print("Processing draft...")
        draft_paragraphs = split_draft(draft)

        draft_modified = RAG(question, draft_paragraphs)

        agent_drafts.append(f"Agent{i+1}: {draft_modified}")

        print(f"[INFO] Agent{i + 1}/{num_agents} retrieved draft...")

        agent_input = '\n\n'.join(agent_drafts) + '\n\n' + refine_prompt

        final_draft = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
                },
                {
                    "role": "user",
                    "content": agent_input
                }
            ],
            temperature=0.5
        ).choices[0].message.content

        print(f"{datetime.now()} - Final draft: {final_draft}")


    return final_draft

# FIX BELOW FUNCTION NOT USING PREVIOUS ANSWER ?? ALSO STRING CHAR LIST WITHIN LIST - SHOULD BE STRING


def get_draft_tot(question, previous_answer, num_agents):

    draft_prompt = """ Base your response on the provided question and the previous answer. Expand the answer by adding more details to enhance its comprehensiveness. Ensure that the expansion maintains logical coherence and enriches the details, making the response more thorough and well-structured.
        Question: {question}
        Previous Answer: {previous_answer}
        IMPORTANT:
        Answer the full question with step-by-step thoughts and make the answer more structural.
        Use `\n\n` to split the answer into several paragraphs.
        Just respond to the instruction directly. DO NOT add additional explanations or introducement in the answer unless you are asked to. """

    refine_prompt = """ 
        Referencing the answers provided by all agents, synthesize a more detailed and comprehensive response by integrating all relevant details from these answers. Ensure logical coherence and provide ONLY THE MERGED ANSWER AS THE OUTPUT, omitting any discussion of the comparison process or analytical thoughts.
"""

    agents_drafts = []
    for i in range(num_agents):
        draft = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
                },
                {
                    "role": "user",
                    "content": draft_prompt
                }
            ],
            temperature=0.5
        ).choices[0].message.content

        draft_paragraphs = split_draft(draft)

        draft_modified = RAG(question, draft_paragraphs)

        agents_drafts.append(f"Agent{i+1}: {draft_modified}")
    
    agents_input = '\n\n'.join(agents_drafts) + '\n\n' + refine_prompt

    final_draft_raw = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
            },
            {
                "role": "user",
                "content": agents_input
            }
        ],
        temperature=0.5
    ).choices[0].message.content

    print(f"##########Final draft raw #########################: {final_draft_raw}...")

    revise_prompt = """
            Based on the original answer and an additional supplementary answer, generate a response that is richer in detail and logically coherent. Review the original answer:
        1. If any part of the answer is correct and requires no further details, retain that portion unchanged and output it directly as it is.
        2. For parts that may be improved or lack necessary details, enhance them by integrating information from the supplementary answer to make the response more comprehensive and accurate.
        3. If you identify any errors within the answers, correct these errors while ensuring that the revised content remains logically coherent.
        Original Answer: {previous_answer}
        Supplementary Answer: {final_draft_raw}

        **IMPORTANT**
        Ensure the revised answer maintains a structured format (multiple paragraphs with subtitles) for better clarity. 
        Separate the paragraphs with `\n\n` characters. Output only the enhanced answer directly, without any extra explanations or a
        nnouncements unless specifically requested."""
    
    final_draft = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
            },
            {
                "role": "user",
                "content": revise_prompt
            }
        ],
        temperature=0.5
    ).choices[0].message.content

    return final_draft





In [180]:
from datetime import datetime


def ratt(question, num_agents):
    step_num = num_steps
    print(f"{datetime.now()} [INFO] Retrieving Step 1 draft...")
    draft = get_draft_tot_initial(question,num_agents)
    print(f"{datetime.now()} [INFO] Step 1 draft returned")
    print(f"##################### DRAFT #######################")
    print(draft)
    print(f"#####################  END  #######################")

    print(f"{datetime.now()} [INFO] Processing draft...")
    draft_paragraphs = split_draft(draft)
    print(f"{datetime.now()} [INFO] Draft split into {len(draft_paragraphs)} parts")

    answer_first_state = RAG(question, draft_paragraphs)

    previous_answer = answer_first_state

    each_step_drafts = [f"Step 1 \n: {previous_answer}"]

    for iteration in range(1, step_num):
        print(f"{datetime.now()} [INFO] Retrieving Step {iteration + 1} draft...")
        draft = get_draft_tot(question, previous_answer, num_agents=num_agents)
        print(f"{datetime.now()} [INFO] Step {iteration + 1} draft returned")
        print(f"##################### DRAFT #######################")
        print(draft)
        print(f"#####################  END  #######################")

        print(f"{datetime.now()} [INFO] Processing draft...")
        draft_paragraphs = split_draft(draft)
        print(f"{datetime.now()} [INFO] Draft split into {len(draft_paragraphs)} parts")

        # filtered_paragraphs = filter_paragraphs(draft_paragraphs, iteration, step_num)
        final_answer = RAG(question, draft_paragraphs)

        each_step_drafts.append(f"Step {iteration + 1} \n: {final_answer}")

        # Update previous_answer for the current iteration's response
        previous_answer = final_answer

    draft_cot = COT_agent(question) # for comparison

    if final_output_mode == 'combine_each_step':
        final_draft = '\n\n'.join(each_step_drafts)
        refine_prompt = f"""
                        Referencing the answers provided by each step, synthesize a more detailed and comprehensive response by integrating all relevant details from these answers. Ensure logical coherence and provide ONLY THE MERGED ANSWER AS THE OUTPUT, omitting any discussion of the comparison process or analytical thoughts.
                        """
        previous_answer = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are ChatGPT, a large language model trained by OpenAI, based on the GPT-4 architecture."
                },
                {
                    "role": "user",
                    "content": final_draft + '\n\n' + refine_prompt
                }
            ],
            temperature=1.0
        ).choices[0].message.content

    return draft_cot, previous_answer








In [181]:
answer_cot, answer_ratt = ratt(chatgpt_prompt, num_agents=1)


2024-08-01 16:41:18.229172 [INFO] Retrieving Step 1 draft...
####################draft 0: # Scientific Rationale for Gamma Secretase in Alzheimer's Disease

## Working Hypothesis

Gamma secretase (GSEC) is a multi-subunit protease complex that cleaves type I transmembrane proteins, including the amyloid precursor protein (APP), leading to the generation of amyloid-beta (Aβ) peptides. The accumulation of Aβ peptides, particularly Aβ42, in the brain is a hallmark of Alzheimer's disease (AD) and is believed to play a crucial role in the pathogenesis of the disease. The working hypothesis is that modulation of GSEC activity could reduce the production of pathogenic Aβ42, thereby mitigating amyloid plaque formation and slowing the progression of AD.

## Unmet Medical Need

Alzheimer's disease is a progressive neurodegenerative disorder with no cure and limited treatment options. Current therapies primarily offer symptomatic relief and do not address the underlying pathophysiology of the dis

/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/3877540748.py:7: RuntimeWarning: coroutine 'RAG' was never awaited
  draft = get_draft_tot_initial(question,num_agents)
Object allocated at (most recent call last):
  File "/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/324958690.py", lineno 110
    draft_modified = RAG(question, draft_paragraphs)


##########Final draft raw #########################: I'm sorry, but I don't have access to the specific answers provided by the agents. If you can share the details or content of their responses, I would be happy to help synthesize a comprehensive answer based on that information....


/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/3877540748.py:25: RuntimeWarning: coroutine 'RAG' was never awaited
  draft = get_draft_tot(question, previous_answer, num_agents=num_agents)
Object allocated at (most recent call last):
  File "/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/324958690.py", lineno 173
    draft_modified = RAG(question, draft_paragraphs)


2024-08-01 16:42:07.009256 [INFO] Step 2 draft returned
##################### DRAFT #######################
I'm sorry, but I cannot fulfill that request without the original answer and the supplementary answer provided. Please share the content you would like me to enhance, and I'll be happy to assist you.
#####################  END  #######################
2024-08-01 16:42:07.009549 [INFO] Processing draft...
2024-08-01 16:42:07.009609 [INFO] Draft split into 1 parts
2024-08-01 16:42:07.009673 [INFO] Retrieving Step 3 draft...
##########Final draft raw #########################: I'm sorry, but it seems that I cannot access or retrieve the specific responses from the agents you mentioned. If you provide me with the answers or details from those agents, I can help synthesize a comprehensive response based on that information....
2024-08-01 16:42:09.415372 [INFO] Step 3 draft returned
##################### DRAFT #######################
I'm sorry, but I cannot assist with that.
##########

/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/3877540748.py:41: RuntimeWarning: coroutine 'RAG' was never awaited
  previous_answer = final_answer
Object allocated at (most recent call last):
  File "/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/3877540748.py", lineno 36
    final_answer = RAG(question, draft_paragraphs)
/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/967180656.py:1: RuntimeWarning: coroutine 'RAG' was never awaited
  answer_cot, answer_ratt = ratt(chatgpt_prompt, num_agents=1)
Object allocated at (most recent call last):
  File "/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/3877540748.py", lineno 17
    answer_first_state = RAG(question, draft_paragraphs)


In [145]:
# First, make sure you have all the necessary imports and have defined all the functions from your original code

# Then, create test cells for each function:

# Test split_draft
def test_split_draft():
    test_draft = "Paragraph 1\n\nParagraph 2\n\nParagraph 3"
    result = split_draft(test_draft)
    assert len(result) == 3, f"Expected 3 paragraphs, got {len(result)}"
    print("split_draft test passed")

test_split_draft()

# Test get_revise_answer
def test_get_revise_answer():
    question = "What is the capital of France?"
    answer = "The capital of France is London."
    retrieved_data = "The capital of France is Paris."
    result = get_revise_answer(question, answer, retrieved_data)
    print(f"Revised answer: {result}")
    assert "Paris" in result, f"Expected 'Paris' in the revised answer"
    print("get_revise_answer test passed")

test_get_revise_answer()

# Test RAG
def test_RAG():
    question = "What are the primary colors?"
    draft_paragraphs = ["The primary colors are red and blue.", "Yellow is also a primary color."]
    result = RAG(question, draft_paragraphs)
    print(f"RAG result: {result}")
    assert "red" in result.lower() and "blue" in result.lower() and "yellow" in result.lower(), f"Expected all primary colors in the result"
    print("RAG test passed")

test_RAG()

# Test get_draft_tot_initial
def test_get_draft_tot_initial():
    question = "Explain the water cycle."
    num_agents = 2
    result = get_draft_tot_initial(question, num_agents)
    print(f"Number of drafts: {len(result)}")
    for i, draft in enumerate(result):
        print(f"Draft {i+1}:\n{draft}\n")
    assert len(result) == 2, f"Expected 2 drafts, got {len(result)}"
    print("get_draft_tot_initial test passed")

test_get_draft_tot_initial()

split_draft test passed
Revised answer: The capital of France is Paris. 


get_revise_answer test passed
Completed iteration 1/2


/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/1479425663.py:50: RuntimeWarning: coroutine 'main' was never awaited
  api_response = main(question, answer, num_queries=2)  # Now using the entire answer instead of just the paragraph
Object allocated at (most recent call last):
  File "/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/1479425663.py", lineno 50
    api_response = main(question, answer, num_queries=2)  # Now using the entire answer instead of just the paragraph


Completed iteration 2/2
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


RESULT OF PUBMED API:
The primary colors are red, blue, and yellow.

In color theory, primary colors are defined as colors that cannot be created by mixing other colors together. They serve as the foundation for creating a wide range of other colors through various combinations.

In the additive color model, which is used in contexts like television and computer screens, the primary colors are red, green, and blue (RGB). By mixing these colors in different ways, a broad spectrum of colors can be produced.

In contrast, the subtractive color model, commonly used in painting and printing, identifies red, blue, and yellow as the primary colors. When mixed together, these colors can create a variety of other hues.

It's important to note that in the subtractive color model, yellow is indeed considered a primary color, while in the additive model, the primary colors are red, green, and

/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/3750429350.py:30: RuntimeWarning: coroutine 'main' was never awaited
  result = RAG(question, draft_paragraphs)
Object allocated at (most recent call last):
  File "/var/folders/02/fvfmdq_j709g1tthj47t5fcm0000gn/T/ipykernel_17808/1479425663.py", lineno 50
    api_response = main(question, answer, num_queries=2)  # Now using the entire answer instead of just the paragraph


####################draft 0: The water cycle, also known as the hydrological cycle, is a continuous process that describes the movement of water within the Earth's atmosphere, land, and bodies of water. The cycle can be broken down into several key stages:

1. **Evaporation**: The process begins when the sun heats up water in rivers, lakes, and oceans. This heat causes the water to evaporate, turning it into water vapor that rises into the atmosphere. Plants also contribute to this process through transpiration, where water is absorbed by roots and released as vapor from leaves.

2. **Condensation**: As water vapor rises, it cools and condenses into tiny droplets, forming clouds. This process occurs when the air temperature drops and the water vapor reaches its dew point, leading to the formation of clouds in the atmosphere.

3. **Precipitation**: Eventually, the water droplets in clouds combine to form larger droplets. When they become heavy enough, they fall back to the Earth's surfa

KeyboardInterrupt: 